In [4]:
import unidecode
import string
import random
import re
import os
import torch
import torch.nn as nn
# add nn.functional module 
import torch.nn.functional as F

from torch.autograd import Variable

In [5]:
file = unidecode.unidecode(open('./datasets/lose_yourself_eminem.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 4063


In [28]:
def random_chunk(chunk_len = 200):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]


print(random_chunk())


bile home, that's when its
Back to the lab again yo, this whole rhapsody
He better go capture this moment and hope it don't pass him
You better lose yourself in the music, the moment
You own it, you be


In [29]:
# Turn string into list of longs
all_characters = string.printable
print(all_characters)

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print('abcDEF is changed to ', char_tensor('abcDEF'))

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

abcDEF is changed to  tensor([10, 11, 12, 39, 40, 41])


In [30]:
def random_training_set():    
    chunk = random_chunk()
    inputs = char_tensor(chunk[:-1])
    targets = char_tensor(chunk[1:])

    return inputs, targets

In [120]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.output_dim = output_dim
        
        self.encoder = nn.Embedding(input_dim, hidden_dim).cuda()
        
        #############
        self.x2h = nn.Linear(self.hidden_dim, self.hidden_dim * 4)
        self.h2h = nn.Linear(self.hidden_dim, self.hidden_dim * 4)
        #############
        
        self.decoder = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, input, cn, hn):
        #############
        
        x = self.encoder(input)
        # input and hidden state encode and concatenate
        gate = self.x2h(x) + self.h2h(hn)
        
        # slice the gate to 4 gate  
        ingate, forgetgate,  cellgate ,outgate = torch.chunk(gate, 4, 2)
        
        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        outgate = F.sigmoid(outgate)
        cellgate = F.tanh(cellgate)
        c_new = torch.mul(cn, forgetgate) + torch.mul(ingate, cellgate)
        h_new = torch.mul(outgate, F.tanh(c_new))
        
        out = self.decoder(h_new)
        
        return out, (c_new, h_new)
        
        #############
    
    def init_hidden(self, batch_size = 1):
        #############
        cn = Variable(torch.zeros(self.layer_dim, batch_size, self.hidden_dim))
        hn = Variable(torch.zeros(self.layer_dim, batch_size, self.hidden_dim))
        return cn, hn

        #############
    
hidden_dim = 100
n_layers = 1
n_characters = len(all_characters)

model = LSTMModel(n_characters, hidden_dim, n_layers, n_characters)
model.cuda()

LSTMModel(
  (encoder): Embedding(100, 100)
  (x2h): Linear(in_features=100, out_features=400, bias=True)
  (h2h): Linear(in_features=100, out_features=400, bias=True)
  (decoder): Linear(in_features=100, out_features=100, bias=True)
)

In [121]:
criterion = nn.CrossEntropyLoss()

lr = 0.005
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [122]:
def evaluate(prime_str='W', predict_len=100, model= None):
    # suppose prims_str is a single character
    # and use greedy search to predict the next character
    cn, hn = model.init_hidden()
    #############
    prime_input = Variable(char_tensor(prime_str))

    hn, cn= hn.cuda(), cn.cuda()
    prime_input = prime_input.cuda()
    predicted = prime_str
    # prime str calculated
    for p in range(len(prime_str)-1):
        _, (cn, hn) = model(prime_input[:,p], cn, hn)

    inp = prime_input.view(1, -1)[:,-1]
    # another str calculated
    for p in range(predict_len):
        output, (cn, hn) = model(inp, cn, hn)
        
        output_dist = output.data.view(-1)
        top_i = torch.argmax(output_dist)
        predicted_char = all_characters[top_i]
        #generate character set
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char)).cuda()
    
    return predicted
    #############

In [123]:
torch.cuda.is_available()

True

In [124]:
n_epochs = 2000
print_every = 100
plot_every = 10

all_losses = []
loss_avg = 0
chunk_len = 200


for epoch in range(1, n_epochs + 1):
    #############
    
    optimizer.zero_grad()
    cn, hn = model.init_hidden(1)
    inp, target = random_training_set()
    # avoid to length collapse assign input length to lenn
    lenn = len(inp)
    inp, target = inp.unsqueeze(0).cuda(), target.unsqueeze(0).cuda()
    cn, hn = cn.cuda(), hn.cuda()
    loss = 0
    # do lstm
    for c in range(lenn):
        out,(cn, hn) = model(inp[:,c], cn, hn)
        loss += criterion(out.view(1, -1), target[:,c])

    loss.backward()
    optimizer.step()
    
    if (epoch+1)%10 == 0:
        all_losses.append(loss.item())
        print("epoch{} average_loss : {}".format(epoch+1,loss.item()))
    if (epoch+1)%100 == 0:
        print('---------------epoch {}-----------------'.format(epoch+1))
        print(evaluate(prime_str = 'I', model = model),'\n')
    
    
        
    #############

epoch10 average_loss : 595.50341796875
epoch20 average_loss : 555.8804321289062
epoch30 average_loss : 471.27947998046875
epoch40 average_loss : 526.01904296875
epoch50 average_loss : 384.5382385253906
epoch60 average_loss : 487.0911865234375
epoch70 average_loss : 356.6087341308594
epoch80 average_loss : 429.94158935546875
epoch90 average_loss : 421.1044616699219
epoch100 average_loss : 351.5999755859375
---------------epoch 100-----------------
I the the the the the the the the the the the the the the the the the the the the the the the the the 

epoch110 average_loss : 461.89111328125
epoch120 average_loss : 340.6536560058594
epoch130 average_loss : 480.2250671386719
epoch140 average_loss : 354.80755615234375
epoch150 average_loss : 231.65545654296875
epoch160 average_loss : 403.0273132324219
epoch170 average_loss : 195.44094848632812
epoch180 average_loss : 375.97021484375
epoch190 average_loss : 307.6420593261719
epoch200 average_loss : 144.65650939941406
---------------epoch 200-

epoch1460 average_loss : 5.4461565017700195
epoch1470 average_loss : 40.118690490722656
epoch1480 average_loss : 43.93147277832031
epoch1490 average_loss : 47.82338333129883
epoch1500 average_loss : 8.861849784851074
---------------epoch 1500-----------------
In yo, this whole rhapsody
He better go capture it
Or just let it slip?
Yo
His palms are sweaty, knee 

epoch1510 average_loss : 8.940126419067383
epoch1520 average_loss : 35.94286346435547
epoch1530 average_loss : 10.770984649658203
epoch1540 average_loss : 32.44501876831055
epoch1550 average_loss : 133.3943634033203
epoch1560 average_loss : 46.996761322021484
epoch1570 average_loss : 36.955291748046875
epoch1580 average_loss : 35.16130065917969
epoch1590 average_loss : 37.89440155029297
epoch1600 average_loss : 54.02006912231445
---------------epoch 1600-----------------
In yo, this whole rhapsody
He better go capture this moment and hope it don', but he keeps on forgett 

epoch1610 average_loss : 38.435997009277344
epoch1620 av

In [125]:
from matplotlib import pyplot as plt
 
a = list(range(200))
plt.plot(a, all_losses)
plt.show()

### Discussion
> Model sturucture : 처음에는 embedding space의 dimension이 헷갈려서 어려움을 겪었지만 linear mapping 이후 수식대로 연산하기 쉽게 dimension 설정이 100x100으로 되어있어서 편하게 하였다. chunk를 이용하여 nn.Linear를 8개 생성하지않고 두 개 생성 이후 slice하여 사용하였다.

> Evaluate : 처음에는 open source등을 참고해 exponential distribution으로 output을 normalize하고 최댓값을 뽑아내는 방식으로 하였는데 그럴 경우 값이 만족스럽게 나오지 않아서 어차피 최댓값을 뽑는 것이면 argmax도 괜찮을 것이다 생각해서 argmax를 사용하니 결과가 훨씬 괜찮게 나왔다.
<img src="https://drive.google.com/uc?id=1pTmhYLmBwaXbIiAkOg0BOnWTNgsMoNt_" title="bad generation" alt="no_image" style="width: 700px;"/>
<img src="https://drive.google.com/uc?id=1MQgERyYAujoFc2A6HXPCD9brk1XigkXc" title="good generation" alt="no_image" style="width: 700px;"/>
> Training : 해당 부분도 처음에 training 할 때 hn과 cn의 아웃풋 위치를 헷갈려서 처음에는 training이 되지 않았다가 수정 후 쉽게 연산이 되었다.